In [ ]:
import os
import cv2
import numpy as np
import glob
import pandas as pd
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred, smooth=0):
    intersection = K.sum(y_true * y_pred, axis=[0,1])
    union = K.sum(y_true, axis=[0,1]) + K.sum(y_pred, axis=[0,1])
    return K.mean((2. * intersection + smooth) / (union + smooth), axis=0)

def Calculate_Score(answer_path, prediction_path):
    answer_path = answer_path
    prediction_path = prediction_path
    answer_list = os.listdir(answer_path)
    prediction_list = os.listdir(prediction_path)
    #print(answer_list)
    #print(prediction_list)
    
    B_score = []
    G_score = []
    R_score = []
    for i in range(len(answer_list)):
        ans = cv2.imread(f'{answer_path}{answer_list[i]}').astype('float32')
        ans[:, :, 0] = np.where(ans[:, :, 0] > 1, 1, 0)
        ans[:, :, 1] = np.where(ans[:, :, 1] > 1, 1, 0)
        ans[:, :, 2] = np.where(ans[:, :, 2] > 1, 1, 0)
        pre = cv2.imread(f'{prediction_path}{prediction_list[i]}').astype('float32')
        pre[:, :, 0] = np.where(pre[:, :, 0] > 1, 1, 0)
        pre[:, :, 1] = np.where(pre[:, :, 1] > 1, 1, 0)
        pre[:, :, 2] = np.where(pre[:, :, 2] > 1, 1, 0)

        
        for j in range(0,3):
            # BGR
            dice = dice_coef(ans[:,:,0+j:1+j], pre[:,:,0+j:1+j])
            dice_score = dice.numpy()
            if np.isnan(dice_score):
                print(prediction_list[i])
            
            if j == 0:
                B_score.append(dice_score)
            elif j == 1:
                G_score.append(dice_score)
            else:
                R_score.append(dice_score)
    print(np.mean(B_score))
    print(np.mean(G_score))
    print(np.mean(R_score))
    print('The final score : ', (np.mean(B_score)+np.mean(G_score)+np.mean(R_score))/3)

In [ ]:
Calculate_Score("train_data/images/ori/","runs/detect/yolov7part1/pre/")